### Bibliotecas

In [29]:
import geopandas as gpd
import pandas as pd
import cx_Oracle
import os
from dotenv import load_dotenv
from shapely.wkt import loads
from shapely import wkt
from funcoes_conexao import conexao_oracle, finaliza_conexao, consulta_para_dataframe, truncate_tabelas

### Diretórios

In [30]:
# DEFINIÇÕES DE DIRETÓRIOS
BASE_DIR = os.getcwd()
DIR_PAI = os.path.dirname(os.path.dirname(BASE_DIR))
DIR_COLETA_DADOS = os.path.join(DIR_PAI, "Coleta de Dados")

# Diretórios de coordenadas
DIR_CORD = os.path.join(DIR_COLETA_DADOS, "COORDENADAS_REGIOES")
DIR_MUNICIPIO_SP = os.path.join(DIR_CORD, "cidade_sp.geojson")
DIR_ZONAS_SP = os.path.join(DIR_CORD, "zonas_sp.geojson")
DIR_DISTRITOS_SP = os.path.join(DIR_CORD, "distrito_sp.geojson")

# Diretórios de dados históricos
DIR_PORTAL_INMET = os.path.join(DIR_COLETA_DADOS, "PORTAL_INMET")
DIR_TRATADOS_CIDADE_SP = os.path.join(DIR_PORTAL_INMET, "DADOS_TRATADOS_CIDADE_SP")

In [31]:
load_dotenv()

True

In [32]:
username = os.getenv('USERNAME_ORC')
password = os.getenv("PASSWORD_ORC")
dsn = 'ORACLE.FIAP.COM.BR:1521/ORCL'

#### Inicializando Conexão ORC

In [33]:
connection, cursor = conexao_oracle(username, password, dsn)

#if connection and cursor:
#    finaliza_conexao(cursor, connection)

#### Limpa Tabelas

In [34]:
truncate_tabelas(connection)

Truncando a tabela TBL_Previsao...
Truncando a tabela TBL_Previsao_Futura...
Truncando a tabela TBL_Distrito...
Truncando a tabela TBL_Zona...
Truncando a tabela TBL_Municipio...
Todas as tabelas foram truncadas com sucesso!
Cursor fechado.


#### INSERT COORDENADAS - TBL_Municipio, TBL_Zona, TBL_Distrito 

##### TBL_Municipio

In [35]:
# Consulta SQL
query_teste = "SELECT * FROM TBL_Municipio"

# Executa a consulta e obtém os resultados em um DataFrame
df_teste = consulta_para_dataframe(cursor, query_teste)
df_teste

,CD_MUN,NM_MUN,SG_ESTADO,AREA_KM2,GEOMETRY,CORD_CENTRAL


In [36]:
# Carregar o arquivo GeoJSON usando GeoPandas
gdf_mun = gpd.read_file(DIR_MUNICIPIO_SP)

# Verifique o conteúdo do GeoDataFrame
print(gdf_mun.head())
print(gdf_mun.dtypes)

    CD_MUN     NM_MUN  AREA_KM2                   CORD_CENTRAL  \
0  3550308  São Paulo   1521.11  POINT (-46.647972 -23.649892)   

                                            geometry  
0  POLYGON ((-46.55066 -23.35714, -46.55011 -23.3...  
CD_MUN             int32
NM_MUN            object
AREA_KM2         float64
CORD_CENTRAL      object
geometry        geometry
dtype: object


In [37]:
def insert_municipio_data(connection, gdf):
    cursor = connection.cursor()

    try:
        for _, row in gdf.iterrows():
            cd_mun = row['CD_MUN']
            nm_mun = row['NM_MUN']
            sg_estado = 'SP'  # Ajuste conforme necessário
            area_km2 = row['AREA_KM2']

            # Certifique-se de que as colunas 'geometry' e 'CORD_CENTRAL' são objetos Shapely
            if isinstance(row['geometry'], str):
                geom = loads(row['geometry'])
            else:
                geom = row['geometry']

            if isinstance(row['CORD_CENTRAL'], str):
                central_geom = loads(row['CORD_CENTRAL'])
            else:
                central_geom = row['CORD_CENTRAL']

            # Converta para WKT para ser utilizado no PL/SQL
            geometry_wkt = geom.wkt
            cor_central_wkt = central_geom.wkt if central_geom else None

            # Bloco PL/SQL para inserir a geometria no Oracle
            plsql = """
            DECLARE
                geom SDO_GEOMETRY;
                central_geom SDO_GEOMETRY;
            BEGIN
                geom := SDO_GEOMETRY(:geometry_wkt, 4326);
                IF :cor_central_wkt IS NOT NULL THEN
                    central_geom := SDO_GEOMETRY(:cor_central_wkt, 4326);
                ELSE
                    central_geom := NULL;
                END IF;
                INSERT INTO TBL_Municipio (CD_MUN, NM_MUN, SG_Estado, AREA_KM2, GEOMETRY, CORD_CENTRAL)
                VALUES (:cd_mun, :nm_mun, :sg_estado, :area_km2, geom, central_geom);
            END;
            """
            cursor.execute(plsql, cd_mun=cd_mun, nm_mun=nm_mun, sg_estado=sg_estado, area_km2=area_km2, 
                           geometry_wkt=geometry_wkt, cor_central_wkt=cor_central_wkt)

        connection.commit()
        print("Dados inseridos com sucesso!")

    except cx_Oracle.DatabaseError as e:
        error, = e.args
        print(f"Erro ao inserir dados: {error.message}")
        connection.rollback()
        print("Alterações desfeitas (rollback) devido ao erro.")

    except Exception as e:
        print(f"Ocorreu um erro inesperado: {str(e)}")
        connection.rollback()
        print("Alterações desfeitas (rollback) devido ao erro.")

    finally:
        if cursor:
            cursor.close()
        print("Cursor fechado.")

In [38]:
# Inserindo os dados
if connection:
    insert_municipio_data(connection, gdf_mun)

Dados inseridos com sucesso!
Cursor fechado.


In [39]:
def read_municipio_data(connection):
    cursor = connection.cursor()
    
    try:
        # Consultar dados da tabela
        cursor.execute("""
            SELECT 
                CD_MUN, 
                NM_MUN, 
                SG_Estado, 
                AREA_KM2, 
                SDO_UTIL.TO_WKTGEOMETRY(GEOMETRY) as geometry_wkt, 
                SDO_UTIL.TO_WKTGEOMETRY(CORD_CENTRAL) as cord_central_wkt 
            FROM TBL_Municipio
        """)
        
        rows = cursor.fetchall()

        # Extrair as colunas dos resultados
        col_names = [desc[0] for desc in cursor.description]

        # Criar um DataFrame com os resultados
        df = pd.DataFrame(rows, columns=col_names)

        # Converter colunas LOB para string e remover as colunas extras
        df['geometry'] = df['GEOMETRY_WKT'].apply(lambda x: loads(x.read()) if isinstance(x, cx_Oracle.LOB) else loads(x))
        df['CORD_CENTRAL'] = df['CORD_CENTRAL_WKT'].apply(lambda x: loads(x.read()) if isinstance(x, cx_Oracle.LOB) else (loads(x) if x else None))

        # Remover colunas WKT que não são mais necessárias
        df.drop(columns=['GEOMETRY_WKT', 'CORD_CENTRAL_WKT'], inplace=True)

        # Criar um GeoDataFrame
        gdf = gpd.GeoDataFrame(df, geometry='geometry')

        return gdf

    except cx_Oracle.DatabaseError as e:
        error, = e.args
        print(f"Erro ao ler dados: {error.message}")

    finally:
        if cursor:
            cursor.close()
        print("Cursor fechado.")

In [40]:
# Ler os dados da tabela
Select_TBL_Municipio = read_municipio_data(connection)
Select_TBL_Municipio

Cursor fechado.


,CD_MUN,NM_MUN,SG_ESTADO,AREA_KM2,geometry,CORD_CENTRAL
0,3550308,São Paulo,SP,1521.11,"POLYGON ((-46.55066 -23.35714, -46.55011 -23.3...",POINT (-46.647972 -23.649892)


##### TBL_Zona 

In [41]:
# Consulta SQL
query_teste = "SELECT * FROM TBL_ZONA"

# Executa a consulta e obtém os resultados em um DataFrame
df_teste = consulta_para_dataframe(cursor, query_teste)
df_teste

,ID,NOME_ZONA,GEOMETRY,CORD_CENTRAL,MUNICIPIO_ID


In [42]:
# Carregar o arquivo GeoJSON usando GeoPandas
gdf_zon = gpd.read_file(DIR_ZONAS_SP)

# Verifique o conteúdo do GeoDataFrame
print(gdf_zon.head())
print(gdf_zon.dtypes)

              Zona                   CORD_CENTRAL   CD_MUN  \
0  Centro Ampliado  POINT (-46.679579 -23.582388)  3550308   
1     Zona Leste 1  POINT (-46.542663 -23.561831)  3550308   
2     Zona Leste 2  POINT (-46.443353 -23.549433)  3550308   
3       Zona Norte   POINT (-46.673093 -23.45335)  3550308   
4         Zona Sul  POINT (-46.708599 -23.813758)  3550308   

                                            geometry  
0  POLYGON ((-46.67421 -23.64222, -46.67437 -23.6...  
1  POLYGON ((-46.57536 -23.6008, -46.57552 -23.60...  
2  POLYGON ((-46.45643 -23.64132, -46.4565 -23.64...  
3  POLYGON ((-46.67805 -23.51367, -46.67863 -23.5...  
4  POLYGON ((-46.60925 -23.90462, -46.60876 -23.9...  
Zona              object
CORD_CENTRAL      object
CD_MUN             int32
geometry        geometry
dtype: object


In [43]:
def insert_zona_data(connection, gdf_zon):
    cursor = connection.cursor()

    try:
        for _, row in gdf_zon.iterrows():
            nome = row['Zona']
            municipio_id = row['CD_MUN']  # Presumindo que CD_MUN refere-se ao MUNICIPIO_ID

            # Certifique-se de que as colunas 'geometry' e 'CORD_CENTRAL' são objetos Shapely
            if isinstance(row['geometry'], str):
                geom = loads(row['geometry'])
            else:
                geom = row['geometry']

            if isinstance(row['CORD_CENTRAL'], str):
                central_geom = loads(row['CORD_CENTRAL'])
            else:
                central_geom = row['CORD_CENTRAL']

            # Converta para WKT para ser utilizado no PL/SQL
            geometry_wkt = geom.wkt
            cor_central_wkt = central_geom.wkt if central_geom else None

            # Bloco PL/SQL para inserir a geometria no Oracle
            plsql = """
            DECLARE
                geom SDO_GEOMETRY;
                central_geom SDO_GEOMETRY;
            BEGIN
                geom := SDO_GEOMETRY(:geometry_wkt, 4326);
                IF :cor_central_wkt IS NOT NULL THEN
                    central_geom := SDO_GEOMETRY(:cor_central_wkt, 4326);
                ELSE
                    central_geom := NULL;
                END IF;
                INSERT INTO TBL_ZONA (NOME_ZONA, GEOMETRY, CORD_CENTRAL, MUNICIPIO_ID)
                VALUES (:nome, geom, central_geom, :municipio_id);
            END;
            """
            cursor.execute(plsql, nome=nome, municipio_id=municipio_id, 
                           geometry_wkt=geometry_wkt, cor_central_wkt=cor_central_wkt)

        connection.commit()
        print("Dados inseridos com sucesso na TBL_ZONA!")

    except cx_Oracle.DatabaseError as e:
        error, = e.args
        print(f"Erro ao inserir dados: {error.message}")
        connection.rollback()
        print("Alterações desfeitas (rollback) devido ao erro.")

    except Exception as e:
        print(f"Ocorreu um erro inesperado: {str(e)}")
        connection.rollback()
        print("Alterações desfeitas (rollback) devido ao erro.")

    finally:
        if cursor:
            cursor.close()
        print("Cursor fechado.")

In [44]:
if connection:
    insert_zona_data(connection, gdf_zon)

Dados inseridos com sucesso na TBL_ZONA!
Cursor fechado.


In [45]:
def read_zona_data(connection):
    cursor = connection.cursor()
    
    try:
        # Consultar dados da tabela TBL_ZONA
        cursor.execute("""
            SELECT 
                ID, 
                NOME_ZONA, 
                MUNICIPIO_ID,
                SDO_UTIL.TO_WKTGEOMETRY(GEOMETRY) as geometry_wkt, 
                SDO_UTIL.TO_WKTGEOMETRY(CORD_CENTRAL) as cord_central_wkt 
            FROM TBL_ZONA
        """)
        
        rows = cursor.fetchall()

        # Extrair as colunas dos resultados
        col_names = [desc[0] for desc in cursor.description]

        # Criar um DataFrame com os resultados
        df = pd.DataFrame(rows, columns=col_names)

        # Converter colunas LOB para string e remover as colunas extras
        df['geometry'] = df['GEOMETRY_WKT'].apply(lambda x: loads(x.read()) if isinstance(x, cx_Oracle.LOB) else loads(x))
        df['CORD_CENTRAL'] = df['CORD_CENTRAL_WKT'].apply(lambda x: loads(x.read()) if isinstance(x, cx_Oracle.LOB) else (loads(x) if x else None))

        # Remover colunas WKT que não são mais necessárias
        df.drop(columns=['GEOMETRY_WKT', 'CORD_CENTRAL_WKT'], inplace=True)

        # Criar um GeoDataFrame
        gdf = gpd.GeoDataFrame(df, geometry='geometry')

        return gdf

    except cx_Oracle.DatabaseError as e:
        error, = e.args
        print(f"Erro ao ler dados: {error.message}")

    finally:
        if cursor:
            cursor.close()
        print("Cursor fechado.")

In [46]:
# Ler os dados da tabela
Select_TBL_Zona = read_zona_data(connection)
Select_TBL_Zona

Cursor fechado.


,ID,NOME_ZONA,MUNICIPIO_ID,geometry,CORD_CENTRAL
0,1,Centro Ampliado,3550308,"POLYGON ((-46.67421 -23.64222, -46.67437 -23.6...",POINT (-46.679579 -23.582388)
1,2,Zona Leste 1,3550308,"POLYGON ((-46.57536 -23.6008, -46.57552 -23.60...",POINT (-46.542663 -23.561831)
2,3,Zona Leste 2,3550308,"POLYGON ((-46.45643 -23.64132, -46.4565 -23.64...",POINT (-46.443353 -23.549433)
3,4,Zona Norte,3550308,"POLYGON ((-46.67805 -23.51367, -46.67863 -23.5...",POINT (-46.673093 -23.45335)
4,5,Zona Sul,3550308,"POLYGON ((-46.60925 -23.90462, -46.60876 -23.9...",POINT (-46.708599 -23.813758)


##### TBL_Distrito

In [47]:
# Consulta SQL
query_teste = "SELECT * FROM TBL_DISTRITO"

# Executa a consulta e obtém os resultados em um DataFrame
df_teste = consulta_para_dataframe(cursor, query_teste)
df_teste

,CD_DIST,NM_DIST,GEOMETRY,CORD_CENTRAL,ZONA_ID


In [48]:
# Carregar o arquivo GeoJSON usando GeoPandas
gdf_dist = gpd.read_file(DIR_DISTRITOS_SP)

In [49]:
# selecionando apenas as colunas de interesse
Select_TBL_Zona_selecao = Select_TBL_Zona[['ID', 'NOME_ZONA']].rename(columns={'ID': 'ID_ZONA'})

In [50]:
# Fazer o merge das duas tabelas usando a coluna 'Zona'
gdf_dist_merged = gdf_dist.merge(Select_TBL_Zona_selecao, on='NOME_ZONA', how='left')

In [51]:
# Verifique o conteúdo do GeoDataFrame
print(gdf_dist_merged.head())
print(gdf_dist_merged.dtypes)

    CD_MUN     NM_MUN    CD_DIST            NM_DIST  \
0  3550308  São Paulo  355030801          Água Rasa   
1  3550308  São Paulo  355030802  Alto de Pinheiros   
2  3550308  São Paulo  355030803         Anhanguera   
3  3550308  São Paulo  355030804         Aricanduva   
4  3550308  São Paulo  355030805        Artur Alvim   

                    CORD_CENTRAL        NOME_ZONA  \
0   POINT (-46.571849 -23.56688)     Zona Leste 1   
1  POINT (-46.711886 -23.547577)  Centro Ampliado   
2  POINT (-46.792944 -23.430469)       Zona Norte   
3  POINT (-46.515679 -23.573925)     Zona Leste 1   
4  POINT (-46.485831 -23.540212)     Zona Leste 1   

                                            geometry  ID_ZONA  
0  POLYGON ((-46.57207 -23.57948, -46.57212 -23.5...        2  
1  POLYGON ((-46.69581 -23.55327, -46.69604 -23.5...        1  
2  POLYGON ((-46.76157 -23.43178, -46.76157 -23.4...        4  
3  POLYGON ((-46.51779 -23.58627, -46.51786 -23.5...        2  
4  POLYGON ((-46.48574 -23.554

In [52]:
def insert_distrito_data(connection, gdf_dist_merged):
    cursor = connection.cursor()

    try:
        for _, row in gdf_dist_merged.iterrows():
            cd_dist = row['CD_DIST']
            nm_dist = row['NM_DIST']
            zona_id = row['ID_ZONA']

            # Certifique-se de que as colunas 'geometry' e 'CORD_CENTRAL' são objetos Shapely
            if isinstance(row['geometry'], str):
                geom = loads(row['geometry'])
            else:
                geom = row['geometry']

            if isinstance(row['CORD_CENTRAL'], str):
                central_geom = loads(row['CORD_CENTRAL'])
            else:
                central_geom = row['CORD_CENTRAL']

            # Converta para WKT para ser utilizado no PL/SQL
            geometry_wkt = geom.wkt
            cor_central_wkt = central_geom.wkt if central_geom else None

            # Bloco PL/SQL para inserir a geometria no Oracle
            plsql = """
            DECLARE
                geom SDO_GEOMETRY;
                central_geom SDO_GEOMETRY;
            BEGIN
                geom := SDO_GEOMETRY(:geometry_wkt, 4326);
                IF :cor_central_wkt IS NOT NULL THEN
                    central_geom := SDO_GEOMETRY(:cor_central_wkt, 4326);
                ELSE
                    central_geom := NULL;
                END IF;
                INSERT INTO TBL_DISTRITO (CD_DIST, NM_DIST, GEOMETRY, CORD_CENTRAL, ZONA_ID)
                VALUES (:cd_dist, :nm_dist, geom, central_geom, :zona_id);
            END;
            """
            cursor.execute(plsql, cd_dist=cd_dist, nm_dist=nm_dist, 
                           geometry_wkt=geometry_wkt, cor_central_wkt=cor_central_wkt,
                           zona_id=zona_id)

        connection.commit()
        print("Dados inseridos com sucesso na tabela TBL_DISTRITO!")

    except cx_Oracle.DatabaseError as e:
        error, = e.args
        print(f"Erro ao inserir dados na TBL_DISTRITO: {error.message}")
        connection.rollback()
        print("Alterações desfeitas (rollback) devido ao erro.")

    except Exception as e:
        print(f"Ocorreu um erro inesperado: {str(e)}")
        connection.rollback()
        print("Alterações desfeitas (rollback) devido ao erro.")

    finally:
        if cursor:
            cursor.close()
        print("Cursor fechado.")

In [53]:
if connection:
    insert_distrito_data(connection, gdf_dist_merged)

Dados inseridos com sucesso na tabela TBL_DISTRITO!
Cursor fechado.


#### INSERT DADOS - TBL_PREVISAO

In [54]:
from funcoes_utilidades import extract_and_map_excel_data, find_nearest_zone, extract_and_convert_coordinates, insert_data_to_tbl_previsao, process_all_excel_files

In [55]:
# Selecionando coordenda central de cada zona
df_zona = gpd.GeoDataFrame(Select_TBL_Zona, geometry='CORD_CENTRAL')

In [56]:
# Processar todos os arquivos Excel no diretório especificado
process_all_excel_files(DIR_TRATADOS_CIDADE_SP, df_zona, connection)

Processando arquivo: c:\Users\Lenovo\OneDrive\Área de Trabalho\Enterprise Challenge\VigilumSP\Coleta de Dados\PORTAL_INMET\DADOS_TRATADOS_CIDADE_SP\Corrected_INMET_SE_SP_A701_SAO PAULO - MIRANTE_01-01-2007_A_31-12-2007.xlsx
8760 registros inseridos com sucesso na tabela TBL_Previsao.
Processando arquivo: c:\Users\Lenovo\OneDrive\Área de Trabalho\Enterprise Challenge\VigilumSP\Coleta de Dados\PORTAL_INMET\DADOS_TRATADOS_CIDADE_SP\Corrected_INMET_SE_SP_A701_SAO PAULO - MIRANTE_01-01-2009_A_31-12-2009.xlsx
8760 registros inseridos com sucesso na tabela TBL_Previsao.
Processando arquivo: c:\Users\Lenovo\OneDrive\Área de Trabalho\Enterprise Challenge\VigilumSP\Coleta de Dados\PORTAL_INMET\DADOS_TRATADOS_CIDADE_SP\Corrected_INMET_SE_SP_A701_SAO PAULO - MIRANTE_01-01-2010_A_31-12-2010.xlsx
8760 registros inseridos com sucesso na tabela TBL_Previsao.
Processando arquivo: c:\Users\Lenovo\OneDrive\Área de Trabalho\Enterprise Challenge\VigilumSP\Coleta de Dados\PORTAL_INMET\DADOS_TRATADOS_CIDADE_